This notebook evaluates the learning fair representations algorithm on COMPAS data
https://proceedings.mlr.press/v28/zemel13.html

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#features selected as per this tutorial: https://github.com/propublica/compas-analysis/blob/master/Compas%20Analysis.ipynb
df = pd.read_csv('COMPAS_preprocessed.csv')

from sklearn.model_selection import train_test_split
col_X = ['race','age', 'c_charge_degree', 'score_text', 'sex',
       'priors_count', 'days_b_screening_arrest', 'decile_score', 'is_recid', 
        'days_in_jail']
X = df[col_X]
y = df['two_year_recid']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

X_pos = X_train[X_train['race']==1]
X_neg = X_train[X_train['race']==0]

y_pos = np.array(y_train[X_train['race']==1])
y_neg = np.array(y_train[X_train['race']==0])

col_X = ['age', 'c_charge_degree', 'score_text', 'sex',
       'priors_count', 'days_b_screening_arrest', 'decile_score', 'is_recid', 
        'days_in_jail']

X_pos = np.array(X_pos[col_X])
X_neg = np.array(X_neg[col_X])

We've imported our dataset, and we only select the columns we need, as well as our target variable: two_year_recid Feature seelction is based on this notebook: https://github.com/propublica/compas-analysis/blob/master/Compas%20Analysis.ipynb

In [ ]:
# Equation 12

def distances(X, v, alpha, N, D, K):
    dists = np.zeros((N, D))
    for n in range(N):
        for i in range(D):
            for k in range(K):
              #print('n i k: ', n, i,k)
              dists[n, k] += (X[n, i] - v[k, i]) * (X[n, i] - v[k, i]) * alpha[i]
    return dists



In [ ]:
# Equation 2&3
def M_nk(dists, N, K):
    M_nk = np.zeros((N, K))
    exp = np.zeros((N, K))
    denom = np.zeros(N)
    for n in range(N):
        for j in range(K):
            exp[n, j] = np.exp(-1 * dists[n, j])
            denom[n] += exp[n, j]
        for j in range(K):
            if denom[n]:
                M_nk[n, j] = exp[n, j] / denom[n]
            else:
                M_nk[n, j] = exp[n, j] / 1e-6
    return M_nk

In [ ]:
# Equation 6
def M_k(M_nk, N, K):
    M_k = np.zeros(K)
    for j in range(K):
        for n in range(N):
            M_k[j] += M_nk[n, j]
        M_k[j] /= N
    return M_k

In [ ]:
# Equation 8&9
def Loss_x(X, M_nk, v, N, D, K):
    x_n_hat = np.zeros((N, D))
    L_x = 0.0
    for n in range(N):
        for i in range(D):
            for k in range(K):
                x_n_hat[n, i] += M_nk[n, k] * v[k, i]
            L_x += (X[n, i] - x_n_hat[n, i]) * (X[n, i] - x_n_hat[n, i])
    return x_n_hat, L_x

In [ ]:
# Equation 10&11
def Loss_y(M_nk, y, w, N, K):
    yhat = np.zeros(N)
    L_y = 0.0
    for n in range(N):
        for k in range(K):
            yhat[n] += M_nk[n, k] * w[k]
        yhat[n] = 1e-6 if yhat[n] <= 0 else yhat[n]
        yhat[n] = 0.999 if yhat[n] >= 1 else yhat[n]
        L_y += -1 * y[n] * np.log(yhat[n]) - (1.0 - y[n]) * np.log(1.0 - yhat[n])
    return yhat, L_y

In [ ]:

# Equation 4 (target function)
def LFR(params, X_pos, X_neg, y_pos, y_neg, K, A_z, A_x, A_y, results=0):   
    #LFR.iters += 1
    N_pos, D = X_pos.shape # protected set
    N_neg, _ = X_neg.shape # non-protected set
    
    #print('PARAMS SHAPE: ', params.shape)
    alpha_pos = params[:D]
    alpha_neg = params[D: 2 * D]
    w = params[2 * D: (2 * D) + K]
    #print(alpha_pos.shape, alpha_neg.shape, w.shape)
    v = np.matrix(params[(2 * D) + K:]).reshape((K, D)) # set of prototypes

    dists_pos = distances(X_pos, v, alpha_pos, N_pos, D, K)
    dists_neg = distances(X_neg, v, alpha_neg, N_neg, D, K)

    M_nk_pos = M_nk(dists_pos, N_pos, K)
    M_nk_neg = M_nk(dists_neg, N_neg, K)

    M_k_pos = M_k(M_nk_pos, N_pos, K)
    M_k_neg = M_k(M_nk_neg, N_neg, K)

    L_z = 0.0
    for k in range(K):
        L_z += abs(M_k_pos[k] - M_k_neg[k]) # Equation 7

    x_n_hat_pos, L_x_pos = Loss_x(X_pos, M_nk_pos, v, N_pos, D, K)
    x_n_hat_neg, L_x_neg = Loss_x(X_neg, M_nk_neg, v, N_neg, D, K)
    L_x = L_x_pos + L_x_neg

    yhat_pos, L_y_pos = Loss_y(M_nk_pos, y_pos, w, N_pos, K)
    yhat_neg, L_y_neg = Loss_y(M_nk_neg, y_neg, w, N_neg, K)
    L_y = L_y_pos + L_y_neg

    criterion = A_x * L_x + A_y * L_y + A_z * L_z

    if results != 0:
        return yhat_pos, yhat_neg, M_nk_pos, M_nk_neg
    else:
        return criterion

We use scikit learn's framework for optimization: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize https://scipy-lectures.org/advanced/mathematical_optimization/auto_examples/plot_non_bounds_constraints.html

Each of the constraints required for the framework to work needs to be programmed s.t. >= 0 for it to be true.

In [ ]:
import scipy
from scipy.optimize import minimize
from numpy import linalg as LA
D = X_pos.shape[1]
K = 5
alpha_pos = np.random.uniform(size=D)
alpha_neg = np.random.uniform(size=D)
w = np.random.uniform(size=K) 
v = np.random.uniform(size=K*D)
params = alpha_pos
for item in [alpha_neg, w, v]:
  params = np.append(params, item.flatten())
res = scipy.optimize.minimize(LFR, 
                        x0=params, 
                        args=(X_pos, X_neg, y_pos, y_neg, K, 1000, 1e-4, 0.1),
                        method='SLSQP')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: overflow encountered in exp
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


In [ ]:
params = res.x

In [21]:
params = [ 5.82069852e-01,  5.02124425e-01,  9.10965658e-01,  2.74610268e-01,
        3.97474821e-01,  8.59815622e-01, -1.23429220e-02,  8.01572071e-01,
        6.20959318e-01,  4.11713157e-01,  1.12769911e-01,  6.35033690e-01,
        4.74356319e-01,  3.19261536e-01,  5.02481813e-01,  9.02021525e-01,
        5.20529878e-01,  2.21460812e-01,  4.10512509e-02,  1.48162758e-01,
        7.60217917e-01,  5.16203490e-01,  2.11645900e-01,  1.22500502e-03,
        8.55725789e-01,  8.68243998e-01,  5.47817586e-01,  9.90537625e-01,
        3.74669836e-01,  2.39921902e-01,  5.35120539e-01,  5.56152285e-01,
        9.91920355e-01,  2.24086371e-01,  5.26454612e-01,  3.54774321e-01,
        1.68337243e-01, -2.53014775e-01,  5.95022381e-01,  4.20818566e-02,
        8.83086285e-01,  5.02413549e-01,  9.78994332e-01,  1.46986557e-01,
        8.49847517e-01,  5.99330128e-01,  2.27207712e-01,  9.78283710e-01,
        5.06313056e-02,  1.25743358e-02,  3.29875819e+00,  7.98292772e-01,
        6.04982755e-01,  5.08084165e-01,  4.49117642e-01,  5.38505770e-01,
        1.35323396e+00,  2.13704460e-01,  3.18681907e-01,  7.39004365e-01,
        5.14488530e-01,  1.35828801e-01,  4.25325165e-01,  1.88144890e-01,
        3.54994983e-01,  3.04144632e-01,  6.37803973e-01,  3.28983126e-01]

In [22]:
D = X_pos.shape[1]
K = 5
alpha_pos = params[:D]
alpha_neg = params[D: 2*D]
w = params[2*D: (2*D) + K]
v = np.array(params[(2*D) + K: ]).reshape((K,D))#np.random.uniform(size=K*D)

Now we evaluate this algorithm based on the 4 metrics: parity, odd equality, accuracy, and calibration

In [23]:
##calculate parity P(Y_hat = 1 | S = 0) = P(Y_hat = 1 | S = 1)

#1) segment test dataset into sensitive and non sensitive
X_pos = X_test[X_test['race']==1]
X_neg = X_test[X_test['race']==0]

y_pos = np.array(y_test[X_test['race']==1])
y_neg = np.array(y_test[X_test['race']==0])

col_X = ['age', 'c_charge_degree', 'score_text', 'sex',
       'priors_count', 'days_b_screening_arrest', 
       'decile_score', 'days_in_jail', 'is_recid']

X_pos = np.array(X_pos[col_X])
X_neg = np.array(X_neg[col_X])

#2) calculate Y_hat for neg & pos
#recover M_nk
N_pos, D = X_pos.shape # protected set
N_neg, _ = X_neg.shape # non-protected set


dists_pos = distances(X_pos, v, alpha_pos, N_pos, D, K)
dists_neg = distances(X_neg, v, alpha_neg, N_neg, D, K)

M_nk_pos = M_nk(dists_pos, N_pos, K)
M_nk_neg = M_nk(dists_neg, N_neg, K)

yhat_pos, lypos = Loss_y(M_nk_pos, y_pos, w, N_pos, K)
yhat_neg, lyneg = Loss_y(M_nk_neg, y_neg, w, N_neg, K)

#3) find y_hat = 1 for both neg and positive
"""FINDING PARITY"""
(len(yhat_neg[yhat_neg > 0.5])/len(yhat_neg), #for non-sensitive attribute #0.584070796460177
  len(yhat_pos[yhat_pos > 0.5])/len(yhat_pos)) #for sensitive attribute #0.6409774436090225

(0.584070796460177, 0.6409774436090225)

#parity (eval #1)
P(Yhat = 1 | sensitive attribute = 1) = 64.1%
P(Yhat = 1 | sensitive attribute = 0) = 58.4%
The algorithm is able to more accurately predict the protected group as y = 1 than the non protected group, meaning it is more likely to identify them as such.

In [24]:
##evaluation metric 2: equality of odds
"""EQUALITY OF ODDS"""

odds_pos = yhat_pos
odds_neg = yhat_neg
#protected: y = 1, then y =0 
print(len(y_pos[(odds_pos > 0.5) & (y_pos > 0.5)]) / len(y_pos[y_pos > 0.5]), len(y_pos[(odds_pos > 0.5) & (y_pos <= 0.5)]) / len(y_pos[y_pos <= 0.5]))
#nonprotected: y = 1, then y =0 
print(len(y_neg[(odds_neg > 0.5) & (y_neg > 0.5)]) / len(y_neg[y_neg > 0.5]), len(y_neg[(odds_neg > 0.5) & (y_neg <= 0.5)]) / len(y_neg[y_neg <= 0.5]))

0.6495412844036698 0.6319845857418112
0.6535433070866141 0.5424528301886793


equality of odds (eval #2)
For when true value of y = 0 the model predicts the sensitive group as y =1 63.2% of the time, and 54.2% accurately for the non-protected group

For when y = 1, the model predicts the sensitive group as y=1 65% of the time, and 65% accurately for the non-protected group

Given this information, we can see that the algorithm is more likely to predict the sensitive group as y =1 even when they are not indeed rescinded. It's also interesting to see that it predicts the sensitive group around the same likelihood regardless of their status. 

In [27]:
"""evaluation metric 3: accuracy"""
yhat_pos1 = yhat_pos
yhat_pos1[yhat_pos1 > 0.5] = 1
yhat_pos1[yhat_pos1 <= 0.5] = 0

yhat_neg1 = yhat_neg
yhat_neg1[yhat_neg1 > 0.5] = 1
yhat_neg1[yhat_neg1 <= 0.5] = 0
(len(yhat_pos1[(yhat_pos1==y_pos)]) + 
 len(yhat_neg1[(yhat_neg1 == y_neg)])) / len(y_test)

0.5195177956371986

# eval metric 3: accuracy
The overall accuracy of the model is 52%

In [ ]:
#cols=['race', 'age', 'c_charge_degree',	'score_text',	'sex'	,'priors_count',
 #      'days_b_screening_arrest',	'decile_score',	'days_in_jail']
X_pos = X_test[X_test['race']==1]
X_neg = X_test[X_test['race']==0]

y_pos = np.array(y_test[X_test['race']==1])
y_neg = np.array(y_test[X_test['race']==0])

col_X = ['race', 'age', 'c_charge_degree', 'score_text', 'sex',
       'priors_count', 'days_b_screening_arrest', 
       'decile_score', 'days_in_jail'] 

X_simulation = np.vstack([X_pos[col_X], X_neg[col_X]])
X_simulation[:,1:]

array([[40.,  1., -1., ..., -1.,  3.,  0.],
       [23.,  0.,  0., ..., -1.,  7.,  0.],
       [29.,  0., -1., ..., -1.,  2.,  1.],
       ...,
       [61.,  1., -1., ..., -4.,  1.,  0.],
       [48.,  0., -1., ..., -1.,  1.,  1.],
       [27.,  0., -1., ..., -1.,  3.,  0.]])

In [ ]:
"""evaluation metric 4: calibration"""

calibration_pos = len(yhat_pos[((yhat_pos > 0.5) & (y_pos > 0.5)) |
    ((yhat_pos <= 0.5) & (y_pos <= 0.5))])/len(yhat_pos)
calibration_neg = len(yhat_neg[((yhat_neg > 0.5) & (y_neg > 0.5)) |
    ((yhat_neg <= 0.5) & (y_neg <= 0.5))])/len(yhat_neg)
calibration_pos, calibration_neg   

(0.5122180451127819, 0.5309734513274337)

#eval 4: calibration
The protected group is 51.2% calibrated, and the non protected group is 53.1% calibrated -- meaning the model is less likely to predict the protected group correctly but by a very small margin